ทดลองทำ text classification ด้วย simple transformer

In [1]:
!pip install datasets
!pip install pythainlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 54.8 MB/s eta 0:00:00


In [2]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 92.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 84.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 93.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 93.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.6/215.6 kB 25.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from datasets import load_dataset
dataset = load_dataset("wisesight_sentiment")

Generating train split:   0%|          | 0/21628 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2404 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2671 [00:00<?, ? examples/s]

In [4]:
import pandas as pd


In [5]:
df = pd.DataFrame(dataset['train'])
df = df.iloc[:5000] #เอามาน้อยจะได้รันไวๆ

In [6]:
df

,texts,category
0,ไปจองมาแล้วนาจา Mitsubishi Attrage ได้หลังสงกร...,1
1,เปิดศักราชใหม่! นายกฯ แถลงข่าวก่อนการแข่งขันศึ...,1
2,บัตรสมาชิกลดได้อีกไหมคับ,1
3,สนใจ new mazda2ครับ,1
4,😍😍,0
...,...,...
4995,วันนี้พบกับบูธของเราที่เซ็นทรัลอีก 1 วันนะคะ.....,1
4996,#LGBT ผมเป็นชายแท้ และสนับสนุน พรบ.คู่ชีวิตครั...,0
4997,กลุ่มเบจโดนซัดบิ๊กมัมแดกเค้กแล้วพลังเต็ม100มาส...,0
4998,มาส่งที่รามอินทราด้วยนะคะ555,1


In [7]:
import re
from pythainlp.util import normalize


In [8]:


emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # Emoticons
        u"\U0001F300-\U0001F5FF"  # Symbols & Pictographs
        u"\U0001F680-\U0001F6FF"  # Transport & Map Symbols
        u"\U0001F1E0-\U0001F1FF"  # Flags (iOS)
                           "]+", flags=re.UNICODE)

In [9]:
 def clean_text(text):

    text = emoji_pattern.sub("", text)

    text = re.sub("#[*a-zA-Zก-๙0-9_]+","", text)
    text = re.sub("[^a-zA-Zก-๙0-9\. ]","", text)

    text = re.sub("5+\+|5{3,}", "555",text)

    text = normalize(text)

    return text

In [10]:
df['cleaned_text'] = df['texts'].apply(clean_text)

In [11]:
df

,texts,category,cleaned_text
0,ไปจองมาแล้วนาจา Mitsubishi Attrage ได้หลังสงกร...,1,ไปจองมาแล้วนาจา Mitsubishi Attrage ได้หลังสงกร...
1,เปิดศักราชใหม่! นายกฯ แถลงข่าวก่อนการแข่งขันศึ...,1,เปิดศักราชใหม่ นายกฯ แถลงข่าวก่อนการแข่งขันศึก...
2,บัตรสมาชิกลดได้อีกไหมคับ,1,บัตรสมาชิกลดได้อีกไหมคับ
3,สนใจ new mazda2ครับ,1,สนใจ new mazda2ครับ
4,😍😍,0,
...,...,...,...
4995,วันนี้พบกับบูธของเราที่เซ็นทรัลอีก 1 วันนะคะ.....,1,วันนี้พบกับบูธของเราที่เซ็นทรัลอีก 1 วันนะคะ.....
4996,#LGBT ผมเป็นชายแท้ และสนับสนุน พรบ.คู่ชีวิตครั...,0,ผมเป็นชายแท้ และสนับสนุน พรบ.คู่ชีวิตครับ เราค...
4997,กลุ่มเบจโดนซัดบิ๊กมัมแดกเค้กแล้วพลังเต็ม100มาส...,0,กลุ่มเบจโดนซัดบิ๊กมัมแดกเค้กแล้วพลังเต็ม100มาส...
4998,มาส่งที่รามอินทราด้วยนะคะ555,1,มาส่งที่รามอินทราด้วยนะคะ555


In [12]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Preparing train data

#train data ต้นฉบับ ลบทิ้งได้เลยเพราะมี data อยู่แล้ว
# train_data = [
#     ["Aragorn was the heir of Isildur", 1],
#     ["Frodo was the heir of Isildur", 0],
#     ["Pippin is stronger than Merry", 2],
# ]




train_df = df[['cleaned_text', 'category']] #pd.DataFrame(train_data)
train_df.columns = ["text", "labels"]

# Preparing eval data ยังไม่ทำ evaluate ก็ปิดไป
# eval_data = [
#     ["Aragorn was the heir of Elendil", 1],
#     ["Sam was the heir of Isildur", 0],
#     ["Merrry is stronger than Pippin", 2],
# ]
# eval_df = pd.DataFrame(eval_data)
# eval_df.columns = ["text", "labels"]


# Optional model configuration
model_args = ClassificationArgs(num_train_epochs=1) #เพื่อความรวดเร็วเลยใช้ epoc 1

# Create a ClassificationModel
model = ClassificationModel(
    'camembert',
    'airesearch/wangchanberta-base-att-spm-uncased',
    num_labels=4, # data set มี label 4 ตัวก็ต้องเปลี่ยนตาม (เช็คที่ df)
    args=model_args
)

# Train the model
model.train_model(train_df)

# Evaluate the model
# result, model_outputs, wrong_predictions = model.eval_model(eval_df)

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:486: UserWarning: use_multiprocessing automatically disabled as camembert fails when using multiprocessing for feature conversion.
  warnings.warn(


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/625 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


(625, 0.8798475738525391)

In [17]:
# Make predictions with the model
predictions, raw_outputs = model.predict(["ดีสุดยอดครับ"])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
predictions

array([0])